In [1]:
# build transaction
from algosdk.future import transaction
from algosdk import constants
from algosdk.v2client import algod
from algosdk import account, mnemonic
import base64
import logging
import sys
from pyteal import *

In [2]:
sys.path.insert(1, '../contracts')
sys.path.append("..")
sys.path.append(".")

from test_contract import approval_program, clear_state_program

In [3]:
def get_private_key_from_mnemonic(mn) :
    private_key = mnemonic.to_private_key(mn)
    return private_key

def compile_program(client, contract):
    teal = compileTeal(contract, mode=Mode.Application, version=5)
    response = client.compile(teal)
    return base64.b64decode(response["result"])


def get_client(addr, token):
    algod_client = algod.AlgodClient(token, addr)
    return algod_client

In [4]:
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
client = get_client(algod_address, algod_token)
sender_addr = "L6ZS626IHSZN7SB6B2WFVW4G6EL2VNKGPP6KQB2FMOQOCRLAZFG3C5BIIY"
sender_pass = "dove tomato solid light purpose shrug shaft blade scrub ten peace bulk \
                net dynamic broken exotic dog knife method august goat drive recycle abandon limb"
sender_private = get_private_key_from_mnemonic(sender_pass)
sender = {"passphrase":sender_pass, "private_key":sender_private, "address": sender_addr}

In [5]:
# create new application
def create_app(client, private_key, approval_program, clear_program, global_schema, local_schema, app_args):
    # define sender as creator
    sender = account.address_from_private_key(private_key)

    # declare on_complete as NoOp
    on_complete = transaction.OnComplete.NoOpOC.real

    # get node suggested parameters
    params = client.suggested_params()
    
    # create unsigned transaction
    txn = transaction.ApplicationCreateTxn(
        sender=sender,
        on_complete=on_complete,
        approval_program=approval_program,
        clear_program=clear_program,
        global_schema=global_schema,
        local_schema=local_schema,
        app_args=app_args,
        sp=params,
    )
    
    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 4)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return

    # display results
    transaction_response = client.pending_transaction_info(tx_id)
    app_id = transaction_response['application-index']
    print("Created new app-id:", app_id)

    return app_id


In [6]:
# helper function that formats global state for printing
def format_state(state):
    formatted = {}
    for item in state:
        key = item['key']
        value = item['value']
        formatted_key = base64.b64decode(key).decode('utf-8')
        if value['type'] == 1:
            # byte string
            if formatted_key == 'voted':
                formatted_value = base64.b64decode(value['bytes']).decode('utf-8')
            else:
                formatted_value = value['bytes']
            formatted[formatted_key] = formatted_value
        else:
            # integer
            formatted[formatted_key] = value['uint']
    return formatted

# helper function to read app global state
def read_global_state(client, app_id):
    app = client.application_info(app_id)
    global_state = app['params']['global-state'] if "global-state" in app['params'] else []
    return format_state(global_state)

In [7]:
# declare application state storage (immutable)
local_ints = 0
local_bytes = 0
global_ints = 1
global_bytes = 10
global_schema = transaction.StateSchema(global_ints, global_bytes)
local_schema = transaction.StateSchema(local_ints, local_bytes)
print("--------------------------------------------")
print("Deploying Counter application......")

# compile program to binary
approval_program_compiled = compile_program(client, approval_program())

# compile program to binary
clear_state_program_compiled = compile_program(client, clear_state_program())

members_list = ["one", "two", "L6ZS626IHSZN7SB6B2WFVW4G6EL2VNKGPP6KQB2FMOQOCRLAZFG3C5BIIY", "6UMSNCZUNWOTWAYRQKE3CV6JXSVLPYA4BMG77CKZM75VASAAOFSLNKTALA"]
# create new application
app_id = create_app(client, sender["private_key"], approval_program_compiled, clear_state_program_compiled, 
                    global_schema, local_schema, members_list)

# read global state of application
print("Global state:", read_global_state(client, app_id))

--------------------------------------------
Deploying Counter application......
TXID:  UM2EHMTBAFWKYHLBBPKBISEMKUSCEWL4WGLNDC2GV7FL3N5QA4HA
Result confirmed in round: 6923
Created new app-id: 16
Global state: {'Creator': 'X7Mva8g8st/IPg6sWtuG8ReqtUZ7/KgHRWOg4UVgyU0=', 'member4': 'NlVNU05DWlVOV09UV0FZUlFLRTNDVjZKWFNWTFBZQTRCTUc3N0NLWk03NVZBU0FBT0ZTTE5LVEFMQQ==', 'member1': 'b25l', 'member2': 'dHdv', 'member3': 'TDZaUzYyNklIU1pON1NCNkIyV0ZWVzRHNkVMMlZOS0dQUDZLUUIyRk1PUU9DUkxBWkZHM0M1QklJWQ=='}


In [8]:
# call application
def call_app(client, private_key, index, app_args) :
    # declare sender
    sender = account.address_from_private_key(private_key)

    # get node suggested parameters
    params = client.suggested_params()

    # create unsigned transaction
    txn = transaction.ApplicationNoOpTxn(sender, params, index, app_args)
    

    # sign transaction
    signed_txn = txn.sign(private_key)
    tx_id = signed_txn.transaction.get_txid()

    # send transaction
    client.send_transactions([signed_txn])

    # wait for confirmation
    try:
        transaction_response = transaction.wait_for_confirmation(client, tx_id, 5)
        print("TXID: ", tx_id)
        print("Result confirmed in round: {}".format(transaction_response['confirmed-round']))

    except Exception as err:
        print(err)
        return
    print("Application called")


In [9]:
sender2_addr = "6UMSNCZUNWOTWAYRQKE3CV6JXSVLPYA4BMG77CKZM75VASAAOFSLNKTALA"
sender2_pass = "select canyon order half resemble clever tissue panther tunnel agent inspire \
            degree system donate gorilla strong vocal divide about magic vessel beef throw ability idea"
sender2_private = get_private_key_from_mnemonic(sender_pass)
sender2 = {"passphrase":sender2_pass, "private_key":sender2_private, "address": sender2_addr}

In [10]:
print("--------------------------------------------")
print("Calling Counter application......")
app_args = ["6UMSNCZUNWOTWAYRQKE3CV6JXSVLPYA4BMG77CKZM75VASAAOFSLNKTALA"]
call_app(client, sender2_private, app_id, app_args)

# read global state of application
print("Global state:", read_global_state(client, app_id))


--------------------------------------------
Calling Counter application......
TXID:  SGWTW5GXIP4462Z6FDMTAPGCD2AUKRZOMVSYCDDG5NIJS2XV25LA
Result confirmed in round: 6925
Application called
Global state: {'member2': 'dHdv', 'member3': 'TDZaUzYyNklIU1pON1NCNkIyV0ZWVzRHNkVMMlZOS0dQUDZLUUIyRk1PUU9DUkxBWkZHM0M1QklJWQ==', 'Count': 1, 'member4': 'NlVNU05DWlVOV09UV0FZUlFLRTNDVjZKWFNWTFBZQTRCTUc3N0NLWk03NVZBU0FBT0ZTTE5LVEFMQQ==', 'Creator': 'X7Mva8g8st/IPg6sWtuG8ReqtUZ7/KgHRWOg4UVgyU0=', 'member1': 'b25l'}
